# Weighted population outcomes

Use the patient population to combine the outcomes from the separate cohorts.

## Notebook setup

In [1]:
import pandas as pd
import numpy as np
import os

import stroke_outcome.outcome_utilities

In [2]:
dir_output = 'output'

## Import data

Patient proportions:

In [3]:
proportions = pd.read_csv(
    os.path.join(dir_output, 'patient_proportions.csv'),
    index_col=0, header=None).squeeze()

In [4]:
proportions

0
haemorrhagic         0.13600
lvo_no_treatment     0.14648
lvo_ivt_only         0.00840
lvo_ivt_mt           0.08500
lvo_mt_only          0.01500
nlvo_no_treatment    0.50252
nlvo_ivt             0.10660
Name: 1, dtype: float64

Calculate some additional proportions:

In [5]:
# Proportion of treated LVO patients:
prop_lvo = 0.0

for key, value in proportions.items():
    if (('lvo' in key) & ('nlvo' not in key) & ('no_treat' not in key)):
        prop_lvo += value

prop_lvo

0.10840000000000001

In [6]:
# Proportion of treated ischaemic patients:
prop_ischaemic_treated = 0.0

for key, value in proportions.items():
    if (('lvo' in key) & ('no_treat' not in key)):
        prop_ischaemic_treated += value

prop_ischaemic_treated

0.21500000000000002

In [7]:
# Proportion of ischaemic patients:
prop_ischaemic = 1.0 - proportions['haemorrhagic']

prop_ischaemic

0.864

Outcomes:

In [8]:
df_results = pd.read_csv(os.path.join(dir_output, 'cohort_outcomes.csv'), index_col=0)

In [9]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34752 entries, Adur 001A to York 024F
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   closest_ivt_time                              34752 non-null  float64
 1   closest_ivt_unit                              34752 non-null  object 
 2   closest_mt_time                               34752 non-null  float64
 3   closest_mt_unit                               34752 non-null  object 
 4   transfer_mt_time                              34752 non-null  float64
 5   transfer_mt_unit                              34752 non-null  object 
 6   mt_transfer_required                          34752 non-null  bool   
 7   ivt_drip_ship                                 34752 non-null  float64
 8   mt_drip_ship                                  34752 non-null  float64
 9   ivt_mothership                                34752 no

mRS distributions:

In [10]:
df_mrs = pd.read_csv(os.path.join(dir_output, 'cohort_mrs_dists.csv'), index_col=0)

In [11]:
df_mrs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34752 entries, Adur 001A to York 024F
Data columns (total 67 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   closest_ivt_time                   34752 non-null  float64
 1   closest_ivt_unit                   34752 non-null  object 
 2   closest_mt_time                    34752 non-null  float64
 3   closest_mt_unit                    34752 non-null  object 
 4   transfer_mt_time                   34752 non-null  float64
 5   transfer_mt_unit                   34752 non-null  object 
 6   mt_transfer_required               34752 non-null  bool   
 7   ivt_drip_ship                      34752 non-null  float64
 8   mt_drip_ship                       34752 non-null  float64
 9   ivt_mothership                     34752 non-null  float64
 10  mt_mothership                      34752 non-null  float64
 11  drip_ship_nlvo_ivt_mrs_dists_0     34752 non-nu

## Mean outcomes

### Combine LVO results

Combine the separate IVT, MT, and IVT and MT cohorts for LVO into one "LVO" group.

In [12]:
for model_name in ['drip_ship', 'mothership']:
    for outcome_name in ['added_utility', 'mrs_less_equal_2', 'mrs_shift', 'added_mrs_less_equal_2']:
        # New column for results:
        col = f'{model_name}_lvo_mix_{outcome_name}'
        
        # Combine results:
        df_results[col] = (
            (df_results[f'{model_name}_lvo_ivt_{outcome_name}'] * proportions['lvo_ivt_only']) +
            (df_results[f'{model_name}_lvo_mt_{outcome_name}'] * proportions['lvo_mt_only']) +
            (df_results[f'{model_name}_lvo_ivt_mt_{outcome_name}'] * proportions['lvo_ivt_mt'])
        )

        # Divide by proportion of LVO patients:
        df_results[col] = df_results[col] / prop_lvo

### Calculate treatment rates based on expected proportions of LVO/nLVO

Add weighted benefit based on proportions of treated LVO and nLVO: Add weighted utility, mrs<=2, and mean_shift based on stroke type and treatment rates.

The expected shift from the no-treatment outcomes is zero for the following groups:
+ Haemorrhagic
+ LVO no treatment
+ nLVO no treatment

so we can skip adding them in the following equation.

In [13]:
for model_name in ['drip_ship', 'mothership']:
    for outcome_name in ['added_utility', 'mrs_less_equal_2', 'mrs_shift', 'added_mrs_less_equal_2']:
        col = f'{model_name}_weighted_{outcome_name}'
        df_results[col] = (
            (df_results[f'{model_name}_nlvo_ivt_{outcome_name}'] * proportions['nlvo_ivt']) +
            (df_results[f'{model_name}_lvo_ivt_{outcome_name}'] * proportions['lvo_ivt_only']) +
            (df_results[f'{model_name}_lvo_mt_{outcome_name}'] * proportions['lvo_mt_only']) +
            (df_results[f'{model_name}_lvo_ivt_mt_{outcome_name}'] * proportions['lvo_ivt_mt'])
        )
        # Remove proportion of non-ischaemic patients:
        df_results[col] = df_results[col] / (1.0 - proportions['haemorrhagic'])

In [14]:
# Show results
df_results.head().T

LSOA,Adur 001A,Adur 001B,Adur 001C,Adur 001D,Adur 001E
closest_ivt_time,17.6,18.7,17.6,17.6,16.5
closest_ivt_unit,BN25BE,BN25BE,BN112DH,BN112DH,BN112DH
closest_mt_time,17.6,18.7,19.8,19.8,19.8
closest_mt_unit,BN25BE,BN25BE,BN25BE,BN25BE,BN25BE
transfer_mt_time,0.0,0.0,31.6,31.6,31.6
...,...,...,...,...,...
drip_ship_weighted_added_mrs_less_equal_2,0.036258,0.036112,0.029515,0.029515,0.029658
mothership_weighted_added_utility,0.033925,0.033791,0.033656,0.033656,0.033656
mothership_weighted_mrs_less_equal_2,0.141066,0.14092,0.140774,0.140774,0.140774
mothership_weighted_mrs_shift,-0.178152,-0.1774,-0.176649,-0.176649,-0.176649


Save to file:

In [15]:
# Save
df_results.to_csv(os.path.join(dir_output, 'cohort_outcomes_weighted.csv'), index_label='lsoa')

In [16]:
summary = df_results.mean(axis=0).T
summary.to_csv(os.path.join(dir_output, 'cohort_outcomes_weighted_summary.csv'), index_label='lsoa')
summary

/tmp/ipykernel_23020/164885061.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  summary = df_results.mean(axis=0).T


closest_ivt_time                               19.548181
closest_mt_time                                34.939310
transfer_mt_time                               27.509447
mt_transfer_required                            0.675098
ivt_drip_ship                                 109.548181
                                                 ...    
drip_ship_weighted_added_mrs_less_equal_2       0.031021
mothership_weighted_added_utility               0.031797
mothership_weighted_mrs_less_equal_2            0.138757
mothership_weighted_mrs_shift                  -0.166331
mothership_weighted_added_mrs_less_equal_2      0.033949
Length: 64, dtype: float64

## mRS distributions

### Load in admissions data

In [17]:
admissions = pd.read_csv('./data/lsoa_predicted_admissions.csv')

In [18]:
admissions.head(3)

,lsoa11cd,lsoa11nm,msoa11cd,msoa11nm,country,LSOA_predicted_admissions
0,E01000001,City of London 001A,E02000001,City of London 001,E,1.569672
1,E01000002,City of London 001B,E02000001,City of London 001,E,1.569672
2,E01000003,City of London 001C,E02000001,City of London 001,E,1.569672


In [19]:
df_mrs = pd.merge(
    df_mrs.reset_index(), admissions[['lsoa11nm', 'LSOA_predicted_admissions']],
    left_on='LSOA', right_on='lsoa11nm', how='left'
)

### Make non-cumulative mRS distributions

In [20]:
cols_to_combine = [c for c in df_mrs.columns if 'dist' in c]
# Remove repeats from all the mRS bands:
cols_each_scen = sorted(list(set(['_'.join(c.split('_')[:-1]) for c in cols_to_combine])))

In [21]:
cols_each_scen

['drip_ship_lvo_ivt_mrs_dists',
 'drip_ship_lvo_ivt_mt_mrs_dists',
 'drip_ship_lvo_mt_mrs_dists',
 'drip_ship_nlvo_ivt_mrs_dists',
 'mothership_lvo_ivt_mrs_dists',
 'mothership_lvo_ivt_mt_mrs_dists',
 'mothership_lvo_mt_mrs_dists',
 'mothership_nlvo_ivt_mrs_dists']

In [22]:
# Make non-cumulative mRS distributions:
for c in cols_each_scen:
    cols_cumsum = [f'{c}_{i}' for i in range(7)]
    cols_noncum = [f'{c}_noncum_{i}' for i in range(7)]

    new_data = df_mrs[cols_cumsum].copy()
    # Take the difference between mRS bands:
    new_data = np.diff(new_data, prepend=0.0, axis=1)
    # Round the values:
    new_data = np.round(new_data, 5)
    # Store:
    df_mrs[cols_noncum] = new_data

### Import reference no-treatment data

In [23]:
mrs_dists, mrs_dists_notes = (
    stroke_outcome.outcome_utilities.import_mrs_dists_from_file())

In [24]:
mrs_dists

,mRS<=0,mRS<=1,mRS<=2,mRS<=3,mRS<=4,mRS<=5,mRS<=6
Stroke type,,,,,,,
pre_stroke_nlvo,0.583,0.746,0.850,0.951,0.993,1.000,1
pre_stroke_lvo,0.408,0.552,0.672,0.838,0.956,1.000,1
no_treatment_lvo,0.050,0.129,0.265,0.429,0.676,0.811,1
no_treatment_nlvo,0.198,0.460,0.580,0.708,0.856,0.918,1
no_effect_nlvo_ivt_deaths,0.196,0.455,0.574,0.701,0.847,0.908,1
no_effect_lvo_ivt_deaths,0.048,0.124,0.255,0.414,0.653,0.783,1
no_effect_lvo_mt_deaths,0.048,0.124,0.255,0.412,0.649,0.779,1
t0_treatment_nlvo_ivt,0.445,0.642,0.752,0.862,0.941,0.967,1
t0_treatment_lvo_ivt,0.140,0.233,0.361,0.522,0.730,0.838,1


In [25]:
mrs_dist_nlvo_no_treatment = mrs_dists.loc['no_treatment_nlvo'].values
mrs_dist_lvo_no_treatment = mrs_dists.loc['no_treatment_lvo'].values

mrs_dist_nlvo_no_treatment_noncum = np.diff(mrs_dist_nlvo_no_treatment, prepend=0.0)
mrs_dist_lvo_no_treatment_noncum = np.diff(mrs_dist_lvo_no_treatment, prepend=0.0)

### Combine patient cohorts

Combine LVO groups:

In [26]:
for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    cols_noncum = [f'{model_name}_lvo_mix_mrs_dists_noncum_{i}' for i in range(7)]
    cols_cum = [f'{model_name}_lvo_mix_mrs_dists_{i}' for i in range(7)]

    # Column names of existing data:
    cols_lvo_ivt = [f'{model_name}_lvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt_mt = [f'{model_name}_lvo_ivt_mt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_mt = [f'{model_name}_lvo_mt_mrs_dists_noncum_{i}' for i in range(7)]
    
    # Weighted sum:
    df_mrs[cols_noncum] = (
        (df_mrs[cols_lvo_ivt].values * proportions['lvo_ivt_only']) +
        (df_mrs[cols_lvo_ivt_mt].values * proportions['lvo_ivt_mt']) +
        (df_mrs[cols_lvo_mt].values * proportions['lvo_mt_only'])
    )
    # Remove the proportion of non-LVO strokes:
    df_mrs[cols_noncum] = df_mrs[cols_noncum] / prop_lvo
    
    # Convert to cumulative probabilities:
    df_mrs[cols_cum] = np.cumsum(df_mrs[cols_noncum], axis=1)

Weighted _treated_ ischaemic population.

We have no mRS distributions for the haemorrhagic strokes. We calculate the weighted population mRS distributions using the data that we do have, and then divide by the proportion of the population that is not haemorrhagic. This rescales the proportions so that all proportions excluding haemorrhagic sum to 1.

In [27]:
for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    cols_noncum = [f'{model_name}_weighted_treated_mrs_dists_noncum_{i}' for i in range(7)]
    cols_cum = [f'{model_name}_weighted_treated_mrs_dists_{i}' for i in range(7)]

    # Column names of existing data:
    cols_nlvo_ivt = [f'{model_name}_nlvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt = [f'{model_name}_lvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt_mt = [f'{model_name}_lvo_ivt_mt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_mt = [f'{model_name}_lvo_mt_mrs_dists_noncum_{i}' for i in range(7)]
    
    # Weighted sum:
    df_mrs[cols_noncum] = (
        (df_mrs[cols_nlvo_ivt].values * proportions['nlvo_ivt']) +
        (df_mrs[cols_lvo_ivt].values * proportions['lvo_ivt_only']) +
        (df_mrs[cols_lvo_ivt_mt].values * proportions['lvo_ivt_mt']) +
        (df_mrs[cols_lvo_mt].values * proportions['lvo_mt_only'])
    )
    # Remove the proportion of haemorrhagic strokes:
    df_mrs[cols_noncum] = df_mrs[cols_noncum] / prop_ischaemic_treated
    
    # Convert to cumulative probabilities:
    df_mrs[cols_cum] = np.cumsum(df_mrs[cols_noncum], axis=1)

Weighted _total_ ischaemic population.

We have no mRS distributions for the haemorrhagic strokes. We calculate the weighted population mRS distributions using the data that we do have, and then divide by the proportion of the population that is not haemorrhagic. This rescales the proportions so that all proportions excluding haemorrhagic sum to 1.

In [28]:
for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    cols_noncum = [f'{model_name}_weighted_mrs_dists_noncum_{i}' for i in range(7)]
    cols_cum = [f'{model_name}_weighted_mrs_dists_{i}' for i in range(7)]

    # Column names of existing data:
    cols_nlvo_ivt = [f'{model_name}_nlvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt = [f'{model_name}_lvo_ivt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_ivt_mt = [f'{model_name}_lvo_ivt_mt_mrs_dists_noncum_{i}' for i in range(7)]
    cols_lvo_mt = [f'{model_name}_lvo_mt_mrs_dists_noncum_{i}' for i in range(7)]
    
    # Weighted sum:
    df_mrs[cols_noncum] = (
        (df_mrs[cols_nlvo_ivt].values * proportions['nlvo_ivt']) +
        (df_mrs[cols_lvo_ivt].values * proportions['lvo_ivt_only']) +
        (df_mrs[cols_lvo_ivt_mt].values * proportions['lvo_ivt_mt']) +
        (df_mrs[cols_lvo_mt].values * proportions['lvo_mt_only']) +
        (mrs_dist_nlvo_no_treatment_noncum * proportions['nlvo_no_treatment']) +
        (mrs_dist_lvo_no_treatment_noncum * proportions['lvo_no_treatment'])
    )
    # Remove the proportion of haemorrhagic strokes:
    df_mrs[cols_noncum] = df_mrs[cols_noncum] / prop_ischaemic
    
    # Convert to cumulative probabilities:
    df_mrs[cols_cum] = np.cumsum(df_mrs[cols_noncum], axis=1)

### Average mRS distributions by admissions

In [29]:
# Remove repeats from all the mRS bands:
cols_each_scen = sorted(list(set(
    [c.split('mrs_dist')[0] + 'mrs_dist' for c in df_mrs.columns if 'mrs_dist' in c]
)))

In [30]:
cols_each_scen

['drip_ship_lvo_ivt_mrs_dist',
 'drip_ship_lvo_ivt_mt_mrs_dist',
 'drip_ship_lvo_mix_mrs_dist',
 'drip_ship_lvo_mt_mrs_dist',
 'drip_ship_nlvo_ivt_mrs_dist',
 'drip_ship_weighted_mrs_dist',
 'drip_ship_weighted_treated_mrs_dist',
 'mothership_lvo_ivt_mrs_dist',
 'mothership_lvo_ivt_mt_mrs_dist',
 'mothership_lvo_mix_mrs_dist',
 'mothership_lvo_mt_mrs_dist',
 'mothership_nlvo_ivt_mrs_dist',
 'mothership_weighted_mrs_dist',
 'mothership_weighted_treated_mrs_dist']

Only make national statistics for areas where mothership and drip and ship are different:

In [31]:
mask = df_mrs['mt_transfer_required']

In [32]:
from statsmodels.stats.weightstats import DescrStatsW  # for mRS dist stats

In [33]:
df_mrs_national = pd.DataFrame()
df_mrs_national_noncum = pd.DataFrame()
df_mrs_national_std = pd.DataFrame()

# Stats:
for c, col in enumerate(cols_each_scen):
    cols_here = [c for c in df_mrs.columns if ((c.startswith(col)) & ('noncum' in c))]

    # Split list of values into one column per mRS band
    # and keep one row per LSOA.
    vals = df_mrs.loc[mask, cols_here].copy()

    # Create stats from these data:
    weighted_stats = DescrStatsW(vals, weights=df_mrs.loc[mask, 'LSOA_predicted_admissions'], ddof=0)
    # Means (one value per mRS):
    means = weighted_stats.mean
    # Standard deviations (one value per mRS):
    stds = weighted_stats.std
    # Cumulative probability from the mean bins:
    cumulatives = np.cumsum(means)
    
    # Store the data in these columns:
    cols_cumulative = [f'mRS<={i}' for i in range(7)]
    cols_noncum = [f'mRS={i}' for i in range(7)]

    # Store in results df:
    val = col.split('_mrs_dist')[0]
    df_mrs_national_noncum.loc[val, cols_noncum] = means
    df_mrs_national.loc[val, cols_cumulative] = cumulatives
    df_mrs_national_std.loc[val, cols_noncum] = stds

In [34]:
df_mrs_national

,mRS<=0,mRS<=1,mRS<=2,mRS<=3,mRS<=4,mRS<=5,mRS<=6
drip_ship_lvo_ivt,0.103557,0.195440,0.327880,0.490122,0.708571,0.823157,1.0
drip_ship_lvo_ivt_mt,0.114716,0.226046,0.371755,0.544526,0.750471,0.850836,1.0
drip_ship_lvo_mix,0.113820,0.223652,0.368348,0.540313,0.747230,0.848694,1.0
drip_ship_lvo_mt,0.114492,0.225885,0.371704,0.544547,0.750516,0.850856,1.0
drip_ship_nlvo_ivt,0.361342,0.589055,0.705083,0.824155,0.921227,0.955152,1.0
drip_ship_weighted,0.182500,0.390153,0.515474,0.653992,0.819884,0.895748,1.0
drip_ship_weighted_treated,0.236545,0.404824,0.535306,0.681046,0.833500,0.901477,1.0
mothership_lvo_ivt,0.097051,0.188086,0.321012,0.483293,0.703836,0.819824,1.0
mothership_lvo_ivt_mt,0.155232,0.276704,0.420797,0.593779,0.783268,0.872474,1.0
mothership_lvo_mix,0.150723,0.269837,0.413064,0.585218,0.777112,0.868394,1.0


In [35]:
df_mrs_national_noncum

,mRS=0,mRS=1,mRS=2,mRS=3,mRS=4,mRS=5,mRS=6
drip_ship_lvo_ivt,0.103557,0.091883,0.132440,0.162241,0.218449,0.114585,0.176843
drip_ship_lvo_ivt_mt,0.114716,0.111330,0.145709,0.172771,0.205944,0.100365,0.149164
drip_ship_lvo_mix,0.113820,0.109832,0.144696,0.171965,0.206917,0.101464,0.151306
drip_ship_lvo_mt,0.114492,0.111393,0.145819,0.172843,0.205969,0.100340,0.149144
drip_ship_nlvo_ivt,0.361342,0.227713,0.116027,0.119072,0.097072,0.033926,0.044848
drip_ship_weighted,0.182500,0.207653,0.125321,0.138518,0.165893,0.075864,0.104252
drip_ship_weighted_treated,0.236545,0.168279,0.130482,0.145740,0.152454,0.067977,0.098523
mothership_lvo_ivt,0.097051,0.091035,0.132926,0.162281,0.220543,0.115988,0.180176
mothership_lvo_ivt_mt,0.155232,0.121472,0.144093,0.172983,0.189488,0.089206,0.127526
mothership_lvo_mix,0.150723,0.119113,0.143227,0.172153,0.191895,0.091282,0.131606


In [36]:
df_mrs_national_std

,mRS=0,mRS=1,mRS=2,mRS=3,mRS=4,mRS=5,mRS=6
drip_ship_lvo_ivt,0.002619,0.000293,0.000223,0.000032,0.000826,0.000546,0.001285
drip_ship_lvo_ivt_mt,0.009830,0.003991,0.001429,0.001293,0.004516,0.003555,0.007289
drip_ship_lvo_mix,0.009244,0.003653,0.001177,0.001112,0.004216,0.003291,0.006774
drip_ship_lvo_mt,0.010367,0.003729,0.000536,0.000805,0.004602,0.003466,0.007233
drip_ship_nlvo_ivt,0.006673,0.002145,0.000390,0.000636,0.001602,0.000744,0.001157
drip_ship_weighted,0.001799,0.000373,0.000144,0.000129,0.000670,0.000474,0.000944
drip_ship_weighted_treated,0.007229,0.001500,0.000579,0.000520,0.002693,0.001906,0.003792
mothership_lvo_ivt,0.005683,0.000897,0.000350,0.000044,0.001879,0.001281,0.003087
mothership_lvo_ivt_mt,0.012266,0.002498,0.000931,0.000322,0.004746,0.003068,0.005774
mothership_lvo_mix,0.011756,0.002374,0.000886,0.000298,0.004524,0.002929,0.005566


## Errors for weighted combinations

Redo the standard deviations to properly propagate the errors from the separate cohorts.

LVO mix:

In [37]:
drip_ship_lvo_stds_before = df_mrs_national_std.loc['drip_ship_lvo_mix'].copy()
mothership_lvo_stds_before = df_mrs_national_std.loc['mothership_lvo_mix'].copy()

In [39]:
stds_to_combine = ['lvo_ivt', 'lvo_mt', 'lvo_ivt_mt']

for model_name in ['drip_ship', 'mothership']:
    # New column names for std:
    ind = f'{model_name}_lvo_mix'

    # Column names of existing data:
    ind_lvo_ivt = f'{model_name}_lvo_ivt'
    ind_lvo_ivt_mt = f'{model_name}_lvo_ivt_mt'
    ind_lvo_mt = f'{model_name}_lvo_mt'
    
    # Weighted sum:
    df_mrs_national.loc[ind] = np.sqrt(
        (df_mrs_national.loc[ind_lvo_ivt].values * (proportions['lvo_ivt_only']  / prop_lvo))**2.0 +
        (df_mrs_national.loc[ind_lvo_ivt_mt].values * (proportions['lvo_ivt_mt']  / prop_lvo))**2.0 +
        (df_mrs_national.loc[ind_lvo_mt].values * (proportions['lvo_mt_only']  / prop_lvo))**2.0
    )

In [43]:
pd.DataFrame(
    np.stack((
        drip_ship_lvo_stds_before,
        df_mrs_national.loc['drip_ship_lvo_mix'],
        mothership_lvo_stds_before,
        df_mrs_national.loc['mothership_lvo_mix']
    ), axis=-1),
    columns=['Drip and ship before', 'Drip and ship after', 'Mothership before', 'Mothership after']
)

,Drip and ship before,Drip and ship after,Mothership before,Mothership after
0,0.009244,0.091689,0.011756,0.123832
1,0.003653,0.180621,0.002374,0.220807
2,0.001177,0.297097,0.000886,0.335981
3,0.001112,0.435239,0.000298,0.474277
4,0.004216,0.600080,0.004524,0.626056
5,0.003291,0.680474,0.002929,0.697605
6,0.006774,0.800011,0.005566,0.800011


Treated population mix:

In [44]:
drip_ship_weighted_treated_stds_before = df_mrs_national_std.loc['drip_ship_weighted_treated'].copy()
mothership_weighted_treated_stds_before = df_mrs_national_std.loc['mothership_weighted_treated'].copy()

In [45]:
p = prop_ischaemic_treated

for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    ind = f'{model_name}_weighted_treated'

    # Column names of existing data:
    ind_nlvo_ivt = f'{model_name}_nlvo_ivt'
    ind_lvo_ivt = f'{model_name}_lvo_ivt'
    ind_lvo_ivt_mt = f'{model_name}_lvo_ivt_mt'
    ind_lvo_mt = f'{model_name}_lvo_mt'
    
    # Weighted sum:
    df_mrs_national.loc[ind] = np.sqrt(
        (df_mrs_national.loc[ind_nlvo_ivt].values * (proportions['nlvo_ivt'] / p))**2.0 +
        (df_mrs_national.loc[ind_lvo_ivt].values * (proportions['lvo_ivt_only'] / p))**2.0 +
        (df_mrs_national.loc[ind_lvo_ivt_mt].values * (proportions['lvo_ivt_mt'] / p))**2.0 +
        (df_mrs_national.loc[ind_lvo_mt].values * (proportions['lvo_mt_only'] / p))**2.0
    )

In [46]:
pd.DataFrame(
    np.stack((
        drip_ship_weighted_treated_stds_before,
        df_mrs_national.loc['drip_ship_weighted_treated'],
        mothership_weighted_treated_stds_before,
        df_mrs_national.loc['mothership_weighted_treated']
    ), axis=-1),
    columns=['Drip and ship before', 'Drip and ship after', 'Mothership before', 'Mothership after']
)    

,Drip and ship before,Drip and ship after,Mothership before,Mothership after
0,0.007229,0.185027,0.013461,0.181823
1,0.001500,0.305930,0.001035,0.307149
2,0.000579,0.380330,0.000818,0.383671
3,0.000520,0.463822,0.000783,0.469512
4,0.002693,0.547873,0.004197,0.553152
5,0.001906,0.584794,0.002404,0.588674
6,0.003792,0.639160,0.004280,0.639160


Total ischaemic population mix:

In [47]:
drip_ship_weighted_stds_before = df_mrs_national_std.loc['drip_ship_weighted'].copy()
mothership_weighted_stds_before = df_mrs_national_std.loc['mothership_weighted'].copy()

In [49]:
p = prop_ischaemic

for model_name in ['drip_ship', 'mothership']:
    # New column names for non-cumulative and cumulative probabilities:
    ind = f'{model_name}_weighted'

    # Column names of existing data:
    ind_nlvo_ivt = f'{model_name}_nlvo_ivt'
    ind_lvo_ivt = f'{model_name}_lvo_ivt'
    ind_lvo_ivt_mt = f'{model_name}_lvo_ivt_mt'
    ind_lvo_mt = f'{model_name}_lvo_mt'
    
    # Weighted sum:
    df_mrs_national.loc[ind] = np.sqrt(
        (df_mrs_national.loc[ind_nlvo_ivt].values * (proportions['nlvo_ivt'] / p))**2.0 +
        (df_mrs_national.loc[ind_lvo_ivt].values * (proportions['lvo_ivt_only'] / p))**2.0 +
        (df_mrs_national.loc[ind_lvo_ivt_mt].values * (proportions['lvo_ivt_mt'] / p))**2.0 +
        (df_mrs_national.loc[ind_lvo_mt].values * (proportions['lvo_mt_only'] / p))**2.0
    )

In [50]:
pd.DataFrame(
    np.stack((
        drip_ship_weighted_stds_before,
        df_mrs_national.loc['drip_ship_weighted'],
        mothership_weighted_stds_before,
        df_mrs_national.loc['mothership_weighted']
    ), axis=-1),
    columns=['Drip and ship before', 'Drip and ship after', 'Mothership before', 'Mothership after']
)    

,Drip and ship before,Drip and ship after,Mothership before,Mothership after
0,0.001799,0.046042,0.003350,0.045245
1,0.000373,0.076128,0.000257,0.076432
2,0.000144,0.094642,0.000203,0.095474
3,0.000129,0.115419,0.000195,0.116835
4,0.000670,0.136334,0.001044,0.137648
5,0.000474,0.145522,0.000598,0.146487
6,0.000944,0.159050,0.001065,0.159050


## Save results

In [51]:
# Save to file:
df_mrs.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted.csv'))

In [52]:
# Save to file:
df_mrs_national.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted_national.csv'))
df_mrs_national_noncum.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted_national_noncum.csv'))
df_mrs_national_std.to_csv(os.path.join(dir_output, 'cohort_mrs_dists_weighted_national_std.csv'))